In [ ]:
import matplotlib.pyplot as plt
import numpy as np

from polya_gamma import PolyaGammaDist

In [ ]:
import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages
import rpy2.robjects.numpy2ri

try:
    bayeslogit = rpackages.importr('BayesLogit')
except:
    utils = rpackages.importr('utils')
    utils.install_packages('BayesLogit')
    bayeslogit = rpackages.importr('BayesLogit')

## Compare Python output against that of R package

In [ ]:
def comparison_hist(samples_1, samples_2, use_log_scale=False):
    if use_log_scale:
        samples_1 = np.log(samples_1)
        samples_2 = np.log(samples_2)
    x_max = max(samples_1.max(), samples_2.max())
    x_min = min(samples_1.min(), samples_2.min())
    bins = np.linspace(x_min, x_max, 51)
    
    plt.hist(samples_1, alpha=.5, bins=bins, density=True)
    plt.hist(samples_2, alpha=.5, bins=bins, density=True)
    for side in ['left', 'top', 'right']:
        plt.gca().spines[side].set_visible(False)
    plt.yticks([])

In [ ]:
n_samples = 10 ** 6
shape = [1., 2.] # BayesLogit apparently requires double
tilt = [.01, 100.]
    # Sqrt of twice the negative tilting parameter, actually
    
pg = PolyaGammaDist()

In [ ]:
plt.figure(figsize=(14, 4.5))
plt.rcParams['font.size'] = 20

for i in range(2):
    
    python_samples = pg.rand_polyagamma(
        np.tile(shape[i], n_samples).astype(np.int), 
        np.tile(tilt[i], n_samples)
    )
    
    # Sample via R package.
    try:
        rpy2.robjects.numpy2ri.activate()
        r_samples = np.array(
            bayeslogit.rpg(n_samples, shape[i], tilt[i])
        )
    except:
        # In case 'numpy2ri.activate()' fails
        r_samples = np.array([
            bayeslogit.rpg(1, shape[i], tilt[i])[0]
            for i in range(n_samples)
        ])
    
    plt.subplot(1, 2, i + 1)
    comparison_hist(r_samples, python_samples, use_log_scale=True)
    
    plt.xlabel('log(tilted stable)')
    if i == 0:
        plt.legend(['from R', 'from Python'], loc=[.65, .7], frameon=False)

plt.show()

## Compare against another R package

In [ ]:
import rpy2.robjects as robjects
import rpy2.robjects.packages as rpackages
import rpy2.robjects.numpy2ri

try:
    pgdraw = rpackages.importr('pgdraw')
except:
    utils = rpackages.importr('utils')
    utils.install_packages('pgdraw')
    pgdraw = rpackages.importr('pgdraw')

In [ ]:
plt.figure(figsize=(14, 4.5))
plt.rcParams['font.size'] = 20

for i in range(2):
    
    python_samples = pg.rand_polyagamma(
        np.tile(shape[i], n_samples).astype(np.int), 
        np.tile(tilt[i], n_samples)
    )
    
    # Sample via R package.
    try:
        rpy2.robjects.numpy2ri.activate()
        r_samples = np.array(
            pgdraw.rcpp_pgdraw(shape[i], tilt[i] * np.ones(n_samples))
        )
    except:
        r_samples = np.array([
            pgdraw.rcpp_pgdraw(shape[i], tilt[i])[0]
            for i in range(n_samples)
        ])
    
    plt.subplot(1, 2, i + 1)
    comparison_hist(r_samples, python_samples, use_log_scale=True)
    
    plt.xlabel('log(tilted stable)')
    if i == 0:
        plt.legend(['from R', 'from Python'], loc=[.65, .7], frameon=False)

plt.show()

## Make sure general and specialized method return same outputs

In [ ]:
n_samples = 10 ** 2
tilt = .1
shape = 1

In [ ]:
seed = 0

pg = PolyaGammaDist(seed)
general_samples = pg.rand_polyagamma(
    np.ones(n_samples, dtype=np.int), 
    tilt * np.ones(n_samples)
)

pg.set_seed(seed)
simplified_samples = pg.rand_unit_shape_polyagamma(
    tilt * np.ones(n_samples)
)

assert np.all(general_samples == simplified_samples)